In [9]:
import networkx as nx
import numpy as np
import plotly.graph_objects as go
import tensorflow as tf

from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure, from_networkx

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [10]:
def plot_graph(G: nx.graph):
    plot =  figure(
    x_range=Range1d(-10.1, 10.1),
    y_range=Range1d(-10.1, 10.1),
    )

    network_graph = from_networkx(G, nx.spring_layout, scale=10, center=(0,0))

    #Set node size and color
    network_graph.node_renderer.glyph = Circle(size=15, fill_color='skyblue')

    #Set edge opacity and width
    network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

    #Add network graph to the plot
    plot.renderers.append(network_graph)
    output_notebook()
    show(plot)

In [11]:
# G = (V, E) with N=V nodes
from re import I


G = nx.from_numpy_matrix(A := np.matrix('''
    1 1 0 1 0; 
    1 1 0 0 1; 
    0 0 1 0 1; 
    1 0 0 1 0; 
    0 1 1 0 1 
'''))


plot_graph(G)

Loading BokehJS ...

In [12]:
D = np.zeros(A.shape)
for i in range(len(A)):
    D[i, i] = np.sum(A[i, :])

In [13]:
# a stochastic latent variable element in NxF matrix Z

# node features summarized in NxD matrix
#  assume 2 features
X = np.matrix('''
    0 1;
    1 0;
    1 1;
    0 0;
    0 1
''')

$$
q(\bold{Z}|\bold{X, A})=\prod_{i=1}^N q(\bold{z}_i|\bold{X, A})
$$

where 
$$
q(\bold{z}_i|\bold{X, A}) = \mathcal{N}(\bold{z}_i|\mu_i, \text{diag}(\sigma_i^2))
$$

Defines the two-layer GCN, weight matrices $W_i$:
$$
\text{GCN}(\bold{X, A}) = \tilde{\bold{A}}\text{ReLU}(\bold{\tilde{A}XW_0})\bold{W}_1
$$

$GCN_\mu$ and $GCN_\sigma$ share first layer parameters $\bold{W}_0$

Also $\bold{\tilde{A}} = D^{1/2}AD^{-1/2}$ (symmetrically normalized adjacency matrix)

In [14]:
# mu = GCN(X, A) matrix of mean vectors
# log(sigma) = GCN(X, A)

Step 1: Generate values as a product between latent variables

In [15]:
# 

Step 2: Learning by optimizing a loss function  

$$
\mathcal{L} = E_{q(Z|X,A)}[logp(A|Z)] - KL[q(Z|X,A)||p(Z)]
$$

In [53]:
import scipy.sparse as sp

# sp.dia_matrix(A.diagonal()[np.newaxis, :, shape=A.shape)

sp.dia_matrix((A.diagonal()[np.newaxis, :], [0]), shape=A.shape)


ValueError: data array must have rank 2

In [20]:
import tensorflow as tf
from dataclasses import dataclass
from tensorflow.keras import Model 
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dropout, Dense, Multiply
import yaml

with open('graph-vae-params.yml', 'r') as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
    print(params)

NUM_FEATURES=1

class GraphConv(Layer):
    def __init__(self, A, input_dim=2, output_dim=params.get('hidden_1'), dropout=params.get('dropout')):
        super(GraphConv, self).__init__()
        # initialize weights

        self.dropout = dropout
        w_init = tf.initializers.GlorotUniform()
        self.w1 = tf.Variable(
            initial_value=w_init(shape=(input_dim, output_dim), dtype=tf.float32),
            trainable=True
        )

    def call(self, inputs):
        x = inputs
        x = Dropout(1-self.dropout)(x)
        x = Multiply()(x, self.w1)

        # not too sure about how this works
        x = tf.sparse_tensor_dense_matmul(A, x)
        return tf.nn.relu(x)


GC = GraphConv(A)
GC(X)

{'num_features': 2, 'epochs': 200, 'hidden_1': 32, 'hidden_2': 16, 'weight_decay': 0.0, 'dropout': 0.0}


ValueError: Exception encountered when calling layer "graph_conv_11" (type GraphConv).

A merge layer should be called on a list of inputs. Received: input_shape=(5, 2) (not a list of shapes)

Call arguments received by layer "graph_conv_11" (type GraphConv):
  • inputs=tf.Tensor(shape=(5, 2), dtype=int64)